In [16]:
import pickle
import re
from nltk.tokenize import TweetTokenizer
from nltk import ngrams
import numpy as np
from sklearn import svm
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer


In [2]:

train_set = np.asarray(pickle.load(open("train_set.p", "rb")))
test_set = np.asarray(pickle.load(open("test_set.p", "rb")))


In [3]:
y_train = train_set[:, 1].astype(np.int_)
train = train_set[:, 0]

y_test = test_set[:, 1].astype(np.int_)
test = test_set[:, 0]


In [4]:
#print(train_set[30:40])

In [5]:
def get_num_of_hastags(tweet):
    return tweet.count("#")


In [6]:
def get_num_of_mult_punctuation(tweet):
    return len(re.findall(r"[?!]{2,}", tweet))


In [7]:
# TODO: remove if not needed

def get_ngrams(tweet, n):
    tknzr = TweetTokenizer()
    tokens = tknzr.tokenize(tweet)
    
    l = []
    for ngram in ngrams(tokens, n):
        l.append(' '.join(str(i) for i in ngram))

    return l


In [8]:
# TODO: rethink, maybe include loop over tweets in functions itself
# TODO: extract features always for train and test like in get_n...

def extract_features(data_set):
    X = np.zeros((len(data_set), 2))    

    for i, tweet in enumerate(data_set): 
        num_hastags = get_num_of_hastags(tweet)
        num_mult_punct = get_num_of_mult_punctuation(tweet)

        X[i] = np.asarray([num_hastags, num_mult_punct])
    
    return X

X_train = extract_features(train)
X_test = extract_features(test)


In [55]:
def transform_to_pos_neg(np_array):
    np_array[np_array < 0] = -1
    np_array[np_array > 0] = 1
    
    return np_array



In [45]:
def eveluate_model(model, X_test):
    prediction = model.predict(X_test)

    acc = metrics.accuracy_score(y_test, prediction)
    print("Accuracy:", acc)

    acc_interval = acc + metrics.accuracy_score(y_test + 1, prediction) \
                   + metrics.accuracy_score(y_test - 1, prediction)
    print("Accuracy for small interval:", acc_interval)  # Accuracy if predicted value is +-1

    acc_dir = metrics.accuracy_score(transform_to_pos_neg(y_test), transform_to_pos_neg(prediction))
    print("Accuracy for right direction:", acc_dir)


In [66]:
# TODO: include n
# TODO: move above extract features again

def get_ngram_vectors(train, test, n):
    #print(train_set[0:5, 0])
    #print(sum([len(tweet) for tweet in train_set[0:5, 0]]))

    vectorizer = CountVectorizer(ngram_range=(1, n), tokenizer=TweetTokenizer().tokenize)
    train_vector = vectorizer.fit_transform(train)
    test_vector = vectorizer.transform(test)

    return train_vector, test_vector

X_train, X_test = get_ngram_vectors(train, test, 3)


In [67]:
lin_svm = svm.LinearSVC()
lin_svm.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [65]:
eveluate_model(lin_svm, X_test)

Accuracy: 0.185
Accuracy for small interval: 0.625
Accuracy for right direction: 0.52
